In [11]:
import pandas as pd
from dw_client import *
from jqdatasdk import *

In [14]:
def get_df_quality(stocks_list,context):
    # 因子【质量】
    # 获取质量因子数据
    ######################################
    # 获取GPOA数据
    def get_df_gpoa(stocks_list,context):
        # 获取上年末的毛利润与上年末的总资产
        q = query(income.operating_profit,balance.code,balance.total_assets).filter(balance.code.in_(stocks_list))
        context_s = pd.Timestamp(context)
        year = context_s.year - 1
        # 获取上年末数据
        df_gpoa = diewei.get_fundamentals(q, statDate = str(year))
        # 计算gpoa
        df_gpoa['gpoa'] = df_gpoa['operating_profit']/df_gpoa['total_assets']
        df_gpoa.index = df_gpoa['code']
        # 删除无用数据
        del df_gpoa['code']
        del df_gpoa['operating_profit']
        del df_gpoa['total_assets']
        return df_gpoa


    # 获取ACC数据
    def get_df_acc(stocks_list,context):
        # 获取上年末的净利润、上年末的经营性现金流和上年末的总资产
        q = query(income.net_profit,cash_flow.net_operate_cash_flow,balance.code,balance.total_assets).filter(balance.code.in_(stocks_list))
        context_s = pd.Timestamp(context)
        year = context_s.year - 1
        # 获取上年末数据
        df_acc = diewei.get_fundamentals(q, statDate = str(year))
        # 计算acc
        df_acc['acc'] = (df_acc['net_profit'] - df_acc['net_operate_cash_flow'])/df_acc['total_assets']
        df_acc.index = df_acc['code']
        # 删除无用数据
        del df_acc['code']
        del df_acc['net_profit']
        del df_acc['net_operate_cash_flow']
        del df_acc['total_assets']
        return df_acc
    ######################################
    # 获取gpoa数据
    df_gpoa = get_df_gpoa(stocks_list,context)
    # 获取acc数据
    df_acc = get_df_acc(stocks_list,context)
    # 合并
    df_quality = pd.concat([df_gpoa,df_acc],axis = 1)
    # 计算质量因子（仿照《巴菲特的阿尔法——来自中国股票市场的实证研究》）
    df_quality['df_gpoa_rank'] = df_quality['gpoa'].rank(ascending = True,method = 'dense')
    df_quality['df_acc_rank'] = df_quality['acc'].rank(ascending = False,method = 'dense')
    df_quality['z_gpoa'] = (df_quality['df_gpoa_rank'] - df_quality['df_gpoa_rank'].mean())/df_quality['df_gpoa_rank'].std()
    df_quality['z_acc'] = (df_quality['df_acc_rank'] - df_quality['df_acc_rank'].mean())/df_quality['df_acc_rank'].std()
    df_quality['quality'] = df_quality['z_gpoa'] + df_quality['z_acc']
    df_quality['z_quality'] = (df_quality['quality'] - df_quality['quality'].mean())/df_quality['quality'].std()
    df_quality = pd.DataFrame(df_quality['z_quality'])

    return df_quality

In [3]:
import pickle

with open('D:/alpha//量化投资//Z_Score_2016_2020.pkl', 'rb') as f:
    fac_data = pickle.load(f)
print('所有因子导入')

所有因子导入


In [6]:
with open('D://alpha//stock_data//zz800_stock_list.pkl', 'rb') as f:
    zz800 = pickle.load(f)

In [15]:
get_df_quality(zz800,'2016-01-04')

,z_quality
code,
000001.XSHE,-1.376940
000002.XSHE,-0.427326
000008.XSHE,-0.988461
000009.XSHE,-0.656097
000012.XSHE,0.189923
000021.XSHE,-0.569768
000027.XSHE,0.237403
000028.XSHE,0.690628
000031.XSHE,-1.540964


In [16]:
fac_data['2016-01-04']

,z_quality
code,
000001.XSHE,-1.311778
000002.XSHE,-0.304648
000006.XSHE,-0.999962
000009.XSHE,-0.587791
000012.XSHE,0.379914
000021.XSHE,-0.437259
000027.XSHE,0.415755
000028.XSHE,0.817173
000030.XSHE,-0.222214


In [17]:
stocks_list = zz800
q = query(income.net_profit,cash_flow.net_operate_cash_flow,balance.code,balance.total_assets).filter(balance.code.in_(stocks_list))

In [19]:
context_s = pd.Timestamp('2016-01-04')
year = context_s.year - 1
# 获取上年末数据
df_gpoa = diewei.get_fundamentals(q, statDate = str(year))

In [20]:
df_gpoa

,net_profit,net_operate_cash_flow,code,total_assets
0,2.186500e+10,-1.826000e+09,000001.XSHE,2.507149e+12
1,2.594944e+10,1.604602e+10,000002.XSHE,6.112956e+11
2,1.899092e+08,-7.246238e+07,000008.XSHE,3.589355e+09
3,9.658984e+08,1.258639e+07,000009.XSHE,1.811490e+10
4,6.398326e+08,1.092833e+09,000012.XSHE,1.548960e+10
5,1.806603e+08,4.904612e+08,000021.XSHE,1.433752e+10
6,2.057693e+09,3.835384e+09,000027.XSHE,5.806743e+10
7,7.873479e+08,1.047979e+09,000028.XSHE,1.321835e+10
8,1.001222e+09,-1.540483e+09,000031.XSHE,5.702255e+10
9,2.271961e+09,-3.610223e+09,000039.XSHE,1.067632e+11
